### EnergyETL

In [7]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [8]:
# read csv file
file_path = "Resources/us_potential_energy.csv"

# create the data frame 
energy_df = pd.read_csv(file_path)
energy_df

,Unnamed: 0,urbanUtilityScalePV_GWh,urbanUtilityScalePV_GW,urbanUtilityScalePV_km2,ruralUtilityScalePV_GWh,ruralUtilityScalePV_GW,ruralUtilityScalePV_km2,rooftopPV_GWh,rooftopPV_GW,CSP_GWh,...,biopowerGaseous_GWh,biopowerGaseous_GW,biopowerGaseous_Tonnes-CH4,geothermalHydrothermal_GWh,geothermalHydrothermal_GW,EGSGeothermal_GWh,EGSGeothermal_GW,hydropower_GWh,hydropower_GW,hydropower_countOfSites
0,Alabama,35850,20,426,3706838,2114,44058,15475.0,12,0,...,1533,0,326186,0,0,535489.0,67.0,4102,0,2435
1,Alaska,166,0,2,8282976,9005,187608,NaN,1,0,...,61,0,13156,15437,1,NaN,NaN,23675,5,3053
2,Arizona,121305,52,1096,11867693,5147,107230,22736.0,14,12544333,...,837,0,178188,8329,1,1239147.0,157.0,1303,0,1958
3,Arkansas,28960,15,332,4986388,2747,57239,8484.0,6,0,...,1063,0,226178,0,0,628621.0,79.0,6093,1,3268
4,California,246008,111,2320,8855917,4010,83549,106411.0,75,8490916,...,15510,1,3300211,130921,16,1344179.0,170.0,30023,6,9692
5,Colorado,43470,19,399,10238083,4514,94046,16162.0,11,9154524,...,1224,0,260470,8953,1,1251657.0,158.0,7789,1,5060
6,Connecticut,7716,4,100,19627,12,256,6616.0,5,0,...,414,0,88227,0,0,56078.0,7.0,922,0,659
7,Delaware,14856,9,189,272332,167,3482,2185.0,1,0,...,385,0,82013,0,0,22813.0,2.0,30,0,25
8,District of Columbia,8,0,0,0,0,0,2490.0,2,0,...,4,0,977,0,0,697.0,0.0,0,0,2
9,Florida,72787,39,830,5137346,2812,58596,63986.0,49,358,...,3693,0,785787,0,0,374161.0,47.0,682,0,493


In [ ]:
# select the columns
energy_df = energy_df["unnamed:0", "urbanUtilityScalePV_GWh","ruralUtilityScalePV_GWh", "" ]